In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR, MSOL
# from holodeck import utils, detstats, plot
from holodeck import utils, detstats, plot
import holodeck.detstats as ds

import hasasia.sensitivity as hsen
import hasasia.sim as hsim

# Appendix: Regular Old Detection Statistic Calculation 
for 1 Parameter Space Sample, but many strain and sky realizations

### A.1 build sam and calculate gws

In [ ]:
dur = 15.0*YR
cad = .2*YR
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
dfobs = np.diff(fobs_edges)
sam = holo.sam.Semi_Analytic_Model() 
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
hc_ss_L5, hc_bg_L5 = sam.ss_gwb(fobs_edges, realize=20, loudest=5)

### A.2 build pta

In [ ]:
npsrs = 40
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
sigmas = np.ones_like(phis)*5.2e-6
# build sim_pta
pulsars = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                    phi=phis, theta=thetas)


### A.3 calculate bg DP

In [ ]:
dp_bg1 = detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg_L5)
dp_bg2 = detstats.detect_bg(thetas, phis, sigmas, fobs, cad, hc_bg_L5)
plt.scatter(dp_bg1, dp_bg2)
print(dp_bg1)
print(dp_bg2)
plt.xlabel('detect_bg_pta')
plt.ylabel('detect_bg')
plt.title(('F,R,L ='+ str(hc_ss_L5.shape)+'\nsigmas=%.2e' %sigmas[0]))


### A.4 build ss sky realizations

In [ ]:
print(hc_ss_L5.shape)

In [ ]:
S = 15
F = len(fobs)
L = len(hc_ss_L5[0,0])
theta_ss = np.random.uniform(0, np.pi, size=F*S*L).reshape(F,S,L)
phi_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
Phi0_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
iota_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)
psi_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)

### A.5 calculate ss DP

In [ ]:
dp_ss1 = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss_L5, hc_bg_L5,
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss)
dp_ss2 = detstats.detect_ss(thetas, phis, sigmas, cad, dur, fobs, dfobs, hc_ss_L5, hc_bg_L5,
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss)

In [ ]:
plt.scatter(dp_ss1, dp_ss2)
print(dp_ss1.shape, utils.stats(dp_ss1))
print(dp_ss2.shape, utils.stats(dp_ss2))
plt.xlabel('detect_ss_pta')
plt.ylabel('detect_ss')
plt.title(('F,R,L ='+ str(hc_ss_L5.shape)+'\nsigmas=%.2e' %sigmas[0]))


In [ ]:
print(dp_ss1.shape) # (R, S)

In [ ]:
def plot_dpbg_dpss(dpbg, dpss, title):
    R = len(dpbg)
    fig, ax = plt.subplots(figsize = (8,4))
    ax.scatter(np.arange(R), dpbg, label='$\gamma_\mathrm{bg}$, avg = %.2e' % np.mean(dpbg), color='blue')
    ax.errorbar(np.arange(R), np.mean(dpss, axis=1), yerr=np.std(dpss, axis=1), capsize=3,
                linestyle='', marker='o',
                color='red', label = '$\gamma_\mathrm{SS}$ (averaged over sky realizations), avg = %.2e' 
                % np.mean(dpss))
    for ss in range(len(dpss[0])):
        ax.scatter(np.arange(R), dpss[:,ss], alpha=0.25)

    fig.legend(bbox_to_anchor=(0.1,0), bbox_transform=fig.transFigure, loc='upper left')
    ax.set_xlabel('Realization')
    ax.set_ylabel('Detection Probability')
    ax.set_yscale('log')
    ax.set_title(title)
    # plt.ylim(0,1)
    return fig

fig = plot_dpbg_dpss(dp_bg1, dp_ss1, title = ('F,R,L ='+ str(hc_ss_L5.shape)+'\nsigmas=%.2e' %sigmas[0]))

## A.6 %timeit
### Full calculation

In [ ]:
%timeit detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss_L5, hc_bg_L5,\
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss) 

In [ ]:
%timeit detstats.detect_ss(thetas, phis, sigmas, cad, dur, fobs, dfobs, hc_ss_L5, hc_bg_L5, \
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss)

In [ ]:
%timeit detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg_L5)
%timeit detstats.detect_bg(thetas, phis, sigmas, fobs, cad, hc_bg_L5)


### Step by Step

In [ ]:
# unitary vectors
%timeit m_hat = ds._m_unitary_vector(theta_ss, phi_ss, psi_ss)
m_hat = ds._m_unitary_vector(theta_ss, phi_ss, psi_ss)
n_hat = ds._n_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
Omega_hat = ds._Omega_unitary_vector(theta_ss, phi_ss) # (3,F,S,L)

In [ ]:
# check pulsar inputs
# get pulsar properties
thetas = np.zeros(len(pulsars))
phis = np.zeros(len(pulsars))
sigmas = np.zeros(len(pulsars))
for ii in range(len(pulsars)):
    thetas[ii] = pulsars[ii].theta
    phis[ii] = pulsars[ii].phi
    sigmas[ii] = np.mean(pulsars[ii].toaerrs)
pi_hat =ds. _pi_unitary_vector(phis, thetas) # (3,P)

## A.7 Vary L
### L = 100

In [ ]:
def calculate_dps(pulsars, hc_ss, hc_bg, S=15):
    F = len(hc_ss)
    R = len(hc_ss[0])
    L = len(hc_ss[0,0])

    # build ss sky realizations        
    theta_ss = np.random.uniform(0, np.pi, size=F*S*L).reshape(F,S,L)
    phi_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    Phi0_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    iota_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    psi_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)

    dp_bg = detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg)
    dp_ss = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg,
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, 
                                iota_ss=iota_ss, psi_ss=psi_ss)
    return dp_bg, dp_ss 

def time_dps(pulsars, hc_ss, hc_bg, S=15):
    F = len(hc_ss)
    R = len(hc_ss[0])
    L = len(hc_ss[0,0])

    # build ss sky realizations        
    theta_ss = np.random.uniform(0, np.pi, size=F*S*L).reshape(F,S,L)
    phi_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    Phi0_ss = np.random.uniform(0,2*np.pi, size=theta_ss.size).reshape(theta_ss.shape)
    iota_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    psi_ss = np.random.uniform(0, np.pi, size = theta_ss.size).reshape(theta_ss.shape)

    %timeit detstats.detect_bg_pta(pulsars, fobs, cad, hc_bg)
    %timeit detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg,\
                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, \
                                iota_ss=iota_ss, psi_ss=psi_ss)


In [ ]:
hc_ss_L100, hc_bg_L100 = sam.ss_gwb(fobs_edges, realize=20, loudest=100)
dp_bg_L100, dp_ss_L100 = calculate_dps(pulsars, hc_ss_L100, hc_bg_L100, S=15)

In [ ]:
fig = plot_dpbg_dpss(dp_bg_L100, dp_ss_L100, 
                     title = ('F,R,L ='+ str(hc_ss_L100.shape)+'\nsigmas=%.2e' %sigmas[0]))

### L = 1

In [ ]:
hc_ss_L1, hc_bg_L1 = sam.ss_gwb(fobs_edges, realize=20, loudest=1)
dp_bg_L1, dp_ss_L1 = calculate_dps(pulsars, hc_ss_L1, hc_bg_L1, S=15)
fig = plot_dpbg_dpss(dp_bg_L1, dp_ss_L1, 
                     title = ('F,R,L ='+ str(hc_ss_L1.shape)+'\nsigmas=%.2e' %sigmas[0]))

## A.8 % Timeit 

In [ ]:
time_dps(pulsars, hc_ss_L1, hc_bg_L1, S=15)

In [ ]:
time_dps(pulsars, hc_ss_L100, hc_bg_L100, S=15)

## A.9 Time Individual SS Steps